In [1]:
#import library
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import  FOAF, DC, RDF, RDFS, OWL, SKOS, NamespaceManager
from rdflib import URIRef, BNode, Literal
import csv

In [69]:
#Build knowledge graph
g = Graph()

nm = NamespaceManager(g)
clean_recipes_list = csv.reader(open('clean_recipes_OpenRefined.csv'))

food = URIRef("http://data.lirmm.fr/ontologies/food#")
containsIngredient = URIRef("http://data.lirmm.fr/ontologies/food#containsIngredient")

ingredient = URIRef("http://data.lirmm.fr/ontologies/food#Ingredient")
ingredient_ns = Namespace("https://kg.knowledgegraph.tech/food2#Ingredient")#namespace prefix is food2
recipe = URIRef("http://data.lirmm.fr/ontologies/food#Recipe")
recipe_ns = Namespace("https://kg.knowledgegraph.tech/food2#Recipe") #namespace prefix is food2
   
    
i = 0 #For ingredients
j = 0 #For recipes
ingredient_idx = {} #For ingredient de-duplication
ingredient_idx_recon = {} #For reconciling

for row in clean_recipes_list:
    if row[2] == 'Recipe Name_refined': #skip the column name line
        continue
        
    if row[2] != '': # skip empty lines for recipe
        recipe_uri = URIRef(recipe_ns + str(j))
        j += 1
        g.add((recipe_uri, RDF.type, recipe))
        g.add((recipe_uri, RDFS.label, Literal(row[2])))  
    

    if row[9] not in ingredient_idx: #if ingredient already created, use existing URI
        ingredient_uri = URIRef(ingredient_ns + str(i))
        ingredient_idx[row[9]] = str(i)
        ingredient_idx_recon[row[0]] = str(i)
        i += 1
    else:
        ingredient_uri = URIRef(ingredient_ns + ingredient_idx[row[9]])
        ingredient_idx_recon[row[0]] = ingredient_idx[row[9]]
                                                               
    g.add((ingredient_uri, RDF.type, ingredient))
    g.add((ingredient_uri, RDFS.label, Literal(row[9])))
    g.add((recipe_uri, containsIngredient, ingredient_uri))


In [70]:
# add reconciling ingredient URI to RDF file
clean_recipes_reconciling = csv.reader(open('clean_recipes_Reconciling.csv'))


for row in clean_recipes_reconciling:
    if row[0] == 'rowno' or row[9] == '' or row[9] == 'URI':
        continue   
    ingredient_uri = URIRef(ingredient_ns + ingredient_idx_recon[str(row[0])])
    g.add((ingredient_uri, OWL.sameAs, URIRef(row[9])))   



In [71]:
# produce the knowledge graph file
g.serialize(destination = "clean_recipes.ttl", format = "turtle")